In [ ]:
# @title **🎮 Colab Cloud Gaming**

# @markdown ---
# @markdown **🖼️ Wallpaper URL:**
# @markdown _Leave blank to use the default background_
background_url = ""  # @param {"type":"string"}

# @markdown **🌐 Website to auto-open:**
# @markdown _Example: `https://www.youtube.com`_
launch_url = ""  # @param {"type":"string"}

# @markdown ---
# @markdown **🖥️ Display Settings**
# @markdown _Example: `1920x1080` @ `60Hz`_
display_resolution = "1920x1080"  # @param {"type":"string"}
refresh_rate = 60  # @param {"type":"integer"}

# @markdown ---
# @markdown **💾 Auto Backup on Exit**
auto_backup = True  # @param {"type":"boolean"}

import os
import subprocess
import sys
import time
import pwd
import re
from tqdm.notebook import tqdm

os.environ["PATH"] += ":/usr/games:/usr/lib/games"

def no_traceback(exctype, value, tb):
    print("⚠️ Unknown error occurred. Try re-running again.")
sys.excepthook = no_traceback

def run(cmd):
    subprocess.run(
        cmd, shell=True, check=True, text=True,
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )

def optional_input(prompt, default="n"):
    try:
        return input(prompt).strip().lower() or default
    except:
        return default

def user_exists(username):
    try:
        pwd.getpwnam(username)
        return True
    except KeyError:
        return False

def backup_user():
    if not os.path.ismount("/content/drive"):
        print("⚠️ Drive not mounted. Skipping backup.")
        return
    print("⚠️ The new backup will overwrite the previous one and old data may be lost.")
    ans = optional_input("💾 Do you want to back up /home/user? (y/n): ", "n")
    if ans == "y":
        run("pkill -KILL -u user || true")
        if not os.path.isdir("/home/user"):
            print("❌ /home/user does not exist. Backup aborted.")
            sys.exit(1)
        try:
            src, dst = "/home/user", "/content/drive/MyDrive/backup.tar.gz"
            def format_size(s):
                for u in ['B', 'KB', 'MB', 'GB', 'TB', 'PB']:
                    if s < 1024:
                        return f"{s:.2f} {u}"
                    s /= 1024
                return f"{s:.2f} PB"
            try: total = int(subprocess.check_output(f'find "{src}" -mindepth 1 \\( -path "*/Downloads" -o -path "*/.config" -o -name "wallpaper.jpg" \\) -prune -o -type f -print0 | du --files0-from=- -cb | tail -n1',shell=True).decode().strip().split()[0])
            except: total = 0
            if not total: return print("❌ No data found.")

            print(f"⏳ Backing up to Google Drive ({format_size(total)}, ~5–30 min)...")

            tar = subprocess.Popen(["tar","-cf","-","-C",src,"--exclude=Downloads","--exclude=.config","--exclude=wallpaper.jpg","."],
                stdout=subprocess.PIPE, stderr=subprocess.DEVNULL)
            pigz = subprocess.Popen(["pigz","-9"], stdin=subprocess.PIPE, stdout=open(dst,"wb"), stderr=subprocess.DEVNULL)
            pbar = tqdm(total=total, unit='B', unit_scale=True, desc="Backing up", smoothing=0.1)

            while chunk := tar.stdout.read(1024*1024):
                pigz.stdin.write(chunk)
                pbar.update(len(chunk))

            tar.stdout.close(); pigz.stdin.close(); tar.wait(); pigz.wait(); pbar.close()
            print("🎉 Backup completed successfully. It may take some time to appear in Google Drive.")
        except:
            print("❌ Backup failed.")
            sys.exit(1)
    else:
        print("❌ Backup cancelled.")

def restore_backup():
    if not os.path.ismount("/content/drive"):
        print("⚠️ Drive not mounted. Skipping restore.")
        return
    if not os.path.exists("/content/drive/MyDrive/backup.tar.gz"):
        print("❌ Backup file not found!")
        sys.exit(1)
    print("⏳ Restoring backup...")
    run("echo 'root:123456' | sudo chpasswd")
    if not user_exists("user"):
        run("sudo useradd -m user")
        run("echo 'user:123456' | sudo chpasswd")
        run("usermod -aG sudo user")
    run("sudo chown -R user:user /home/user")
    run("pigz -dc /content/drive/MyDrive/backup.tar.gz | tar -xvf - -C /home/user")
    print("🎉 Restore completed successfully.")

def get_tailscale_ip():
    try:
        ip = subprocess.check_output("tailscale ip -4", shell=True, text=True).strip()
        return ip
    except subprocess.CalledProcessError:
        print("⚠️ Unable to get Tailscale IP, fallback to localhost.")
        return "127.0.0.1"

def get_tailscale_nearest_region():
    try:
        output = subprocess.check_output("tailscale netcheck", shell=True, text=True)
        match = re.search(r"Nearest DERP:\s*(\S+)", output)
        if match:
            region = match.group(1)
            print(f"Connected Tailscale region: {region}")
            return region
        else:
            print("⚠️ Could not find nearest DERP region.")
    except subprocess.CalledProcessError:
        print("⚠️ Failed to run tailscale netcheck.")

def uptime_loop():
    start = time.time()
    while True:
        with open("log.txt", "w") as f:
            f.write(f"{int(time.time() - start)} seconds\n")
        time.sleep(30)

def main():
    global display_resolution, refresh_rate
    print("🎮 Starting Colab GPU Gaming Environment...")
    if auto_backup and not os.path.ismount("/content/drive"):
        try:
            from google.colab import drive
            print("⏳ Mounting Google Drive...")
            drive.mount("/content/drive", force_remount=True)
        except Exception as e:
            print(f"⚠️ Cannot mount Google Drive: {e}")
    print("⏳ Preparing installation files. Please wait...")
    if not os.path.exists("/packages") or not os.path.exists("/packages/NVIDIA-Linux-x86_64-550.54.15.run"):
        run("wget -qO- https://github.com/OkamuraYuji/Colab-Gaming/releases/download/1.0.0/packages.tar.gz | pigz -dc | tar -xv -C /")
        run("echo '#!/bin/sh\nexport PATH=\"$PATH:/usr/games:/usr/lib/games\"' | sudo tee /etc/profile.d/custom_path.sh > /dev/null && sudo chmod +x /etc/profile.d/custom_path.sh")
        run("curl -L https://pkgs.tailscale.com/stable/tailscale_1.90.9_amd64.tgz | sudo tar --strip-components=1 -xzv -C /usr/local/bin")
        run("mkdir -p /var/lib/tailscale")
        run("nohup bash -c \"while true; do TS_DEBUG_ALWAYS_USE_DERP=true tailscaled --tun=userspace-networking --socket=/run/tailscale/tailscaled.sock --port 41641 ; sleep 1; done\" &")

    get_tailscale_nearest_region()
    !tailscale up
    tailscale_ip = get_tailscale_ip()
    print(f"Tailscale IP: {tailscale_ip}")

    print("Default password for root and user is: 123456")

    if os.path.exists("/content/drive"):
        if optional_input("💾 Do you want to restore the backup? (y/n): ", "n") == "y":
            restore_backup()
        else:
            print("Skipped restore.")

    try:
        run("nvidia-smi")
    except subprocess.CalledProcessError:
        print("⚠️ NVIDIA-SMI not found or not working. Exiting.")
        sys.exit(1)

    print("⏳ Installing... (~2 minutes)")

    if not os.path.exists("/home/user/wallpaper.jpg"):
        run("rm -f /packages/libsystemd0_249.11-0ubuntu3.16_i386.deb")
        run("rm -f /packages/libssl3_3.0.2-0ubuntu1.19_i386.deb")
        run("rm -f /packages/libapparmor1_3.0.4-2ubuntu2.4_i386.deb")
        run("dpkg --add-architecture i386; apt update")
        run("rm -f /usr/share/doc/*/changelog.Debian.gz")
        run("dpkg --add-architecture i386 && sudo DEBIAN_FRONTEND=noninteractive dpkg -i /packages/*.deb ; sudo DEBIAN_FRONTEND=noninteractive apt-get install --fix-broken -y -o Dir::Cache::archives='/packages'")
        run("sudo apt install pv tint2 -y")

    run("echo \"mode: off\" > ~/.xscreensaver")
    run("chmod +x /packages/NVIDIA*.run && echo 1 | /packages/NVIDIA*.run --no-kernel-module --ui=none")
    run("mkdir -p ~/.config/sunshine")
    run("echo 'root:123456' | sudo chpasswd")

    if not user_exists("user"):
        run("sudo useradd -m user")
        run("echo 'user:123456' | sudo chpasswd")
        run("usermod -aG sudo user")

    if not display_resolution.strip():
        display_resolution = "1920x1080"

    run("nvidia-xconfig -a --allow-empty-initial-configuration "
        f"--virtual={display_resolution} --busid PCI:0:4:0")

    run("nohup sudo Xorg :1 -seat seat-1 -allowMouseOpenFail -novtswitch -nolisten tcp &")
    run("sleep 2")
    run("DISPLAY=:1 xhost +local:")

    try:
        run(f"DISPLAY=:1 xrandr --output DVI-D-0 --mode {display_resolution} --rate {refresh_rate}")
    except subprocess.CalledProcessError:
        print(f"⚠️ Failed to set resolution {display_resolution}, retrying with default 1920x1080.")
        run("DISPLAY=:1 xrandr --output DVI-D-0 --mode 1920x1080 --rate 60")

    if background_url:
        try:
            subprocess.run(f"wget -q --show-progress -O /home/user/wallpaper.jpg {background_url}", shell=True, check=True, text=True)
            print("🎉 Wallpaper downloaded successfully.")
        except subprocess.CalledProcessError:
            print("❌ Failed to download wallpaper. Using default.")
            run("cp /packages/wallpaper.jpg /home/user/")
    else:
        run("cp /packages/wallpaper.jpg /home/user/")

    run("su - user -c \"nohup pulseaudio --exit-idle-time=-1 &\"")
    run("su - user -c \"DISPLAY=:1 nohup openbox &\"")
    run("su - user -c \"DISPLAY=:1 feh --bg-max ~/wallpaper.jpg\"")
    run("su - user -c \"rm -rf ~/.config/sunshine ;DISPLAY=:1 nohup sunshine &\"")
    run("su - user -c \"DISPLAY=:1 nohup tint2 &\"")

    chrome_cmd = f"DISPLAY=:1 nohup google-chrome --url {launch_url} &" if launch_url else "DISPLAY=:1 nohup google-chrome &"
    run(f"su - user -c \"{chrome_cmd}\"")
    run("su - user -c \"DISPLAY=:1 nohup thunar &\"")
    run("su - user -c \"DISPLAY=:1 nohup heroic &\"")

    from flask import Flask, request, render_template_string
    import socket, threading
    import logging
    logging.getLogger("werkzeug").setLevel(logging.ERROR)

    def start_web_interface():
        app = Flask(__name__)

        html_form = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>🎮 Moonlight PIN Setup</title>
            <style>
                body {{ background-color: #111; color: #eee; font-family: Arial; text-align: center; margin-top: 10%; }}
                input {{ padding: 10px; font-size: 18px; border-radius: 6px; border: none; width: 200px; text-align: center; }}
                button {{ padding: 10px 20px; font-size: 18px; background: #28a745; color: white; border: none; border-radius: 6px; cursor: pointer; }}
                button:hover {{ background: #218838; }}
                .container {{ background: #222; display: inline-block; padding: 40px; border-radius: 10px; }}
                #message {{ margin-top: 20px; font-size: 18px; display: none; }}
            </style>
            <script>
                async function submitForm(event) {{
                    event.preventDefault();
                    const pin = document.querySelector('input[name="pin"]').value.trim();
                    const msg = document.getElementById('message');
                    if (!pin) {{
                        msg.style.display = "block";
                        msg.style.color = "red";
                        msg.innerText = "❌ Please enter PIN!";
                        return;
                    }}

                    msg.style.display = "block";
                    msg.style.color = "#ccc";
                    msg.innerText = "⏳ Sending PIN...";

                    const response = await fetch("/", {{
                        method: "POST",
                        headers: {{ "Content-Type": "application/x-www-form-urlencoded" }},
                        body: "pin=" + encodeURIComponent(pin)
                    }});

                    const result = await response.text();
                    if (result.includes("SUCCESS")) {{
                        msg.style.color = "lime";
                        msg.innerText = "🎉 PIN sent successfully!";
                    }} else {{
                        msg.style.color = "red";
                        msg.innerText = "❌ Error sending PIN.";
                    }}
                }}
            </script>
        </head>
        <body>
            <div class="container">
                <h2>🔑 Moonlight PIN</h2>
                <h3>{tailscale_ip}</h3>
                <form onsubmit="submitForm(event)">
                    <input type="text" name="pin" maxlength="4" placeholder="****" required><br><br>
                    <button type="submit">Send PIN</button>
                </form>
                <div id="message"></div>
            </div>
        </body>
        </html>
        """

        @app.route('/', methods=['GET', 'POST'])
        def index():
            if request.method == 'POST':
                pin = request.form['pin'].strip()
                curl_passwd = (
                    f'curl -u admin:admin -X POST -k https://localhost:47990/api/password '
                    '-H "Content-Type: application/json" '
                    '-d \'{"currentUsername":"admin","currentPassword":"admin","newUsername":"admin","newPassword":"admin","confirmNewPassword":"admin"}\''
                )
                curl_pin = (
                    f'curl -u admin:admin -X POST -k https://localhost:47990/api/pin '
                    f'-H "Content-Type: application/json" '
                    f'-d \'{{"pin":"{pin}","name":"my-moonlight-device"}}\''
                )
                run(curl_passwd)
                run(curl_pin)
                return "SUCCESS"
            return render_template_string(html_form)

        app.run(host='0.0.0.0', port=3000, debug=False, use_reloader=False)

    def is_port_in_use(port, host='127.0.0.1'):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            return s.connect_ex((host, port)) == 0

    if not is_port_in_use(3000):
        threading.Thread(target=start_web_interface, daemon=True).start()

    print(f"Web interface: http://{tailscale_ip}:3000 (Access via Tailscale)")
    print("🎉 Setup completed!")

    uptime_loop()

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("Ctrl+C detected.")
        if auto_backup:
            backup_user()
        print("Exiting program...")
